In [62]:
import pandas as pd
import numpy as np
import os

In [44]:
# the folder of gene expression which contains gene expression matrix and parsed data
gene_expression_folder = "../kim_data/gene_expression_matrix/"
GENE_MATRIX = "gene_expression_matrix.csv"
FEATURE_PARSE = "feature_parse_matrix.csv"
gene_matrix_file = os.path.join(gene_expression_folder, GENE_MATRIX)
feature_parse_file = os.path.join(gene_expression_folder, FEATURE_PARSE)

In [45]:
# extract gene efficient length from parsed file
feature_parse_df = pd.read_csv(feature_parse_file)
gene_length_df = feature_parse_df[['Geneid', 'Length']]
gene_length_df.set_index('Geneid', inplace=True)
# transform read counts to tpm
gene_matrix_df = pd.read_csv(gene_matrix_file, index_col='Geneid')
# reset index 

In [157]:
def cnt2tpm(gene_df, length_df):
    merged_df = pd.merge(length_df, gene_df, left_index=True,right_index=True)
    merged_df.iloc[:,1:] = merged_df.iloc[:,1:].div(merged_df['Length'], axis=0)
    #merged_df.iloc[:,1:] = merged_df.iloc[:,1:].div(merged_df.sum()[1:])  * (10**6)
    #merged_df.drop('Length',axis = 1,inplace=True)
    tpm_df = merged_df.iloc[:,1:].div(merged_df.sum()[1:])  * (10**6)
    return tpm_df

In [158]:
def cnt2fpkm(gene_df, length_df):
    merged_df = pd.merge(length_df, gene_df, left_index=True,right_index=True)
    fpkm_df = merged_df.iloc[:,1:].div(merged_df['Length'], axis=0).div(merged_df.sum()[1:]) * (10**9)
    return fpkm_df

In [56]:
gene_length_df_test = gene_length_df.iloc[100:105,]

In [68]:
gene_length_df_test.loc[:,'Length'] = np.array([1,2,3,4,5])

In [69]:
gene_length_df_test

,Length
Geneid,
ENSG00000240731.1,1
ENSG00000224051.2,2
ENSG00000169962.4,3
ENSG00000107404.13,4
ENSG00000162576.12,5


In [70]:
gene_matrix_df_test.loc[:,'SRR2431428'] = np.array([1,2,3,4,5])

In [144]:
gene_matrix_df_test

,SRR2431428,SRR2431433
Geneid,,
ENSG00000240731.1,1,0
ENSG00000224051.2,2,5
ENSG00000169962.4,3,0
ENSG00000107404.13,4,2
ENSG00000162576.12,5,0
